In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score,precision_score,f1_score,recall_score,cohen_kappa_score,multilabel_confusion_matrix,make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import cross_val_score,cross_validate

In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()
df_data.head()

,CONTENT_NEW,WORDCUT,SCORE,LEVEL,Tokens,level1_tokens,level2_tokens,level3_tokens,level4_tokens,level5_tokens,level6_tokens,level7_tokens,char_num,high_level,primary_level,Connective_ratio,connective_SCORE,pre,connective_numbers
0,记得当年年纪小，家中兄弟姐妹数十多人。我排行倒最第二。父母亲为了养起这头家，日夜往外奔波劳累...,记得 当年 年纪 小 ， 家中 兄弟姐妹 数十 多人 。 我 排行 倒 最 第二 。 父母亲...,90,3,342,0,2,7,4,2,0,3,636,5,13,0.067251,2.984263,记得 当年 年纪 小 家中 兄弟姐妹 数十 多人 我 排行 倒 最 第二 父母亲 为了 养起...,23
1,《一封求职信》×××揭阳县西门外磐东镇肇沟乡广东省李文先生快乐家庭旅游公司负责人广东省李文先...,《 一封 求职信 》 × × × 揭阳县 西 门外 磐 东镇肇沟 乡 广东省 李文 先生 快...,80,3,194,0,0,0,3,0,0,5,403,5,3,0.051546,2.434391,一封 求职信 揭阳县 西 门外 磐 东镇肇沟 乡 广东省 李文 先生 快乐 家庭 旅游 公司...,10
2,我的外婆对我的影响非常大。从小，我父母便将我交给外婆带。他们是上班一族，平日无法照顾我。虽然...,我 的 外婆 对 我 的 影响 非常 大 。 从小 ， 我 父母 便 将 我 交给 外婆 带...,80,3,302,0,4,5,5,3,0,4,570,7,14,0.072848,3.791400,我 的 外婆 对 我 的 影响 非常 大 从小 我 父母 便 将 我 交给 外婆 带 他们 ...,22
3,“安乐死”究竟可不可取？在二十世纪的今天，“安乐死”已不是什么新话题。在不同的国家，已曾有人...,“ 安乐死 ” 究竟 可不 可取 ？ 在 二十世纪 的 今天 ， “ 安乐死 ” 已 不是 ...,95,3,273,1,0,4,3,1,1,8,588,10,8,0.084249,3.982972,安乐死 究竟 可不 可取 在 二十世纪 的 今天 安乐死 已 不是 什么 新 话题 在 不同...,23
4,关于安乐死这个问题，近来引起了广泛的讨论，有些人认为：一个人既既身罹不治之症，与其留在世上饱...,关于 安乐死 这个 问题 ， 近来 引起 了 广泛 的 讨论 ， 有些 人 认为 ： 一个 ...,85,3,186,1,1,5,4,1,0,4,364,5,11,0.096774,2.989940,关于 安乐死 这个 问题 近来 引起 了 广泛 的 讨论 有些 人 认为 一个 人 既 既身...,18


In [3]:
# Generating 'y' for classification as well as regression
y = df_data['SCORE']

In [4]:

# generating x 
pre = df_data['pre']
# Tokenization
def preprocessing_sentence(x):
    words = jieba.cut(str(x).strip())
    return ' '.join(words)

In [5]:
#ngrams Vectorizer
print(pre.shape)
cv = CountVectorizer(ngram_range=(1, 3), max_features=1000)
ngram_vector = cv.fit_transform(pre)
print(ngram_vector.shape)


(3668,)
(3668, 1000)


In [6]:
# 特征合并 feature concatenate 
feature_set1 = df_data[['connective_SCORE',]]
feature_set2 = ngram_vector
print(feature_set1.shape)
print(feature_set2.shape)
train_features = sparse.hstack((feature_set2, feature_set1))
print('=============')
print(train_features.shape)

(3668, 1)
(3668, 1000)
(3668, 1001)


In [7]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(train_features, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(2934, 1001)
(2934,)


In [8]:
#Tf-IDF + LR 
#评估
# （准确率、精确度、召回率、F1值、Kappa系数用于评估分类任务）
## 二分类问题，一般设为两类，正类/反类，或0、1. 所以有tp，fn，fp，tn。 t，f代表是否正确，p，n代表是正类还是反类
## 准确率（accuracy）：所有分类正确的样本数，除以样本总数）A= （tp+tn）/（tp+tn+fp+fn）
## 精确度（precision）精确度也叫查准率，判为正的为p，包括正确tp，和错误fp，其中tp即预测正确的比例 P= tp/tp+fp
## 召回率（recall） ： 查全率，判断正确的且为正的是tp，判断错误且为负（该正却负的），想知道tp占实际正确的比例 r= tp/tp+fn
## f1 越大越好，f1=2*tp/(2*tp+fp+fn)
## kappa 衡量分类精度，其计算基于混淆矩阵，目前用的是二次加权kappa，对错误的惩罚更重，应用更广泛
# 逻辑回归LR
lr = LogisticRegression()
lr.fit(x_train, y_train)
train_score = lr.score(x_train, y_train)
y_pred = lr.predict(x_test)
print("Train Accuracy:", train_score)
print("================================================")
y_pred = lr.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)


Train Accuracy: 0.9713701431492843
Test Accuracy: 0.23024523160762944
acc_for_each_class:
 [0.         0.         0.         0.18181818 0.14736842 0.19444444
 0.36       0.29090909 0.15       0.12121212 0.28070175 0.44444444]
qwkappa:
 0.4934967994157322
Weighted precision 0.23077810950462607
Weighted recall 0.23024523160762944
Weighted f1-score 0.22764560457061417
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         5
          45       0.00      0.00      0.00         9
          50       0.00      0.00      0.00        20
          55       0.18      0.17      0.17        48
          60       0.15      0.16      0.16        85
          65       0.19      0.22      0.21        95
          70       0.36      0.34      0.35       157
          75       0.29      0.31      0.30       104
          80       0.15      0.16      0.15        77
          85       0.12      0.12      0.12        65
          90

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# randomforest classfier
rf = ensemble.RandomForestClassifier()
params = {'n_estimators': [50], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
train_score = rf.score(x_train, y_train)
test_score = accuracy_score(y_test, y_pred)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)



Train Accuracy: 0.9996591683708248
Test Accuracy: 0.2683923705722071
acc_for_each_class:
 [0.         0.         0.         0.08333333 0.29032258 0.20560748
 0.2924282  0.32258065 0.26315789 0.21875    0.27272727 1.        ]
qwkappa:
 0.4862810881842069
Weighted precision 0.2713237762680798
Weighted recall 0.2683923705722071
Weighted f1-score 0.22171465087295417
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         5
          45       0.00      0.00      0.00         9
          50       0.00      0.00      0.00        20
          55       0.08      0.04      0.06        48
          60       0.29      0.21      0.24        85
          65       0.21      0.23      0.22        95
          70       0.29      0.71      0.41       157
          75       0.32      0.10      0.15       104
          80       0.26      0.13      0.17        77
          85       0.22      0.11      0.14        65
          90   

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are

In [10]:
# support vector machine classfier
model = OneVsRestClassifier(svm.SVC(kernel='rbf',gamma='scale'))#linear
clt = model.fit(x_train,y_train)
y_pred = clt.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
train_score = clt.score(x_train, y_train)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)

Train Accuracy: 0.9935241990456715
Test Accuracy: 0.25340599455040874
acc_for_each_class:
 [0.5        0.         0.         0.2        0.18181818 0.16470588
 0.32624113 0.31034483 0.19607843 0.21428571 0.25       0.25      ]
qwkappa:
 0.5234786953900823
Weighted precision 0.23565931218981803
Weighted recall 0.25340599455040874
Weighted f1-score 0.22980122574615647
classification report: 
               precision    recall  f1-score   support

          40       0.50      0.40      0.44         5
          45       0.00      0.00      0.00         9
          50       0.00      0.00      0.00        20
          55       0.20      0.17      0.18        48
          60       0.18      0.16      0.17        85
          65       0.16      0.15      0.16        95
          70       0.33      0.59      0.42       157
          75       0.31      0.17      0.22       104
          80       0.20      0.13      0.16        77
          85       0.21      0.18      0.20        65
          90

In [11]:
# 交叉验证 cross validation   logistic regression
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(lr, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

{'fit_time': array([0.77871847, 0.79944134, 0.76577067, 0.87449384, 0.82387185,
       0.83175254, 0.84361839, 0.80493212, 0.8043108 , 0.92487311]), 'score_time': array([0.        , 0.        , 0.        , 0.        , 0.0009985 ,
       0.        , 0.        , 0.        , 0.00099564, 0.00202537]), 'test_accuracy': array([0.21768707, 0.16666667, 0.19727891, 0.2244898 , 0.2116041 ,
       0.17406143, 0.221843  , 0.20819113, 0.22525597, 0.21843003]), 'test_qwkappa': array([0.52614537, 0.40524777, 0.32262988, 0.49967206, 0.41504798,
       0.41901953, 0.52382662, 0.42741474, 0.52167977, 0.5094864 ])}
Mean accuracy: 0.2065508114508602
Mean qwkappa: 0.4570170131881898


D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# random forest classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(rf, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([3.05672026, 3.07450509, 2.79311776, 3.01151681, 2.70615125,
       2.64042497, 2.89749503, 3.02075434, 3.00178528, 3.06069255]), 'score_time': array([0.03091598, 0.02029252, 0.02026272, 0.02393556, 0.01977897,
       0.02026582, 0.02589798, 0.0201726 , 0.0202713 , 0.02490139]), 'test_accuracy': array([0.29251701, 0.24829932, 0.26530612, 0.25170068, 0.23208191,
       0.2559727 , 0.25938567, 0.25938567, 0.28327645, 0.29692833]), 'test_qwkappa': array([0.48196464, 0.43946134, 0.44346973, 0.32160969, 0.44369278,
       0.35327652, 0.41418287, 0.36000794, 0.50864665, 0.48154244])}
Mean accuracy: 0.264485384597525
Mean qwkappa: 0.42478546096395114


In [13]:
# support machine classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(clt, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([27.02580047, 27.3972199 , 25.65024781, 26.38883996, 27.41537142,
       28.87731266, 29.85189939, 28.37330413, 27.70096231, 28.80699205]), 'score_time': array([2.91128564, 2.72042584, 2.47179818, 2.69716477, 2.69400811,
       2.96673727, 2.61925387, 2.69744706, 2.95669103, 2.5125854 ]), 'test_accuracy': array([0.29251701, 0.23469388, 0.2755102 , 0.30272109, 0.2559727 ,
       0.25255973, 0.23890785, 0.27645051, 0.26962457, 0.27303754]), 'test_qwkappa': array([0.55288353, 0.48951593, 0.45071625, 0.50415805, 0.38741483,
       0.46259721, 0.5043332 , 0.45905361, 0.58221772, 0.50886641])}
Mean accuracy: 0.26719950778946394
Mean qwkappa: 0.490175675557272
